In [1]:
import pandas as pd

data = pd.read_csv("../../data/processed/twitter15_dataset_with_tvt.csv", lineterminator="\n")
print(data.shape)
data.head()

(1490, 7)


tweet_id                                         tweet_text  \
0  731166399389962242  🔥ca kkk grand wizard 🔥 endorses @hillaryclinto...   
1  714598641827246081  an open letter to trump voters from his top st...   
2  691809004356501505  america is a nation of second chances —@potus ...   
3  693204708933160960  brandon marshall visits and offers advice, sup...   
4  551099691702956032  rip elly may clampett: so sad to learn #beverl...   

        label       tvt  cv_fold        tt        tvt2  
0  unverified  training        1  training    testting  
1  unverified  training        1      test    training  
2   non-rumor  training        2  training  validation  
3   non-rumor  training        1  training    training  
4        true  training        3  training    testting

In [2]:
length = data.shape[0]
length

1490

In [3]:
import random

# random.seed(37)

train_w = [0 for i in range(675)]
val_w = [1 for i in range(225)]
test_w = [2 for i in range(100)]

weights = train_w + val_w + test_w
print(f"weights : {len(weights)}")

tvt = []
for i in range(length):
    gacha = random.sample(weights, 1)[0]
    if gacha == 0:
        tvt.append("training")
    elif gacha == 1:
        tvt.append("validation")
    else:
        tvt.append("testting")

print(f"Training : {tvt.count('training')} - {round(tvt.count('training')/length, 3)}")
print(f"Validation : {tvt.count('validation')} - {round(tvt.count('validation')/length, 3)}")
print(f"Testing : {tvt.count('testting')} - {round(tvt.count('testting')/length, 3)}")

weights : 1000
Training : 981 - 0.658
Validation : 342 - 0.23
Testing : 167 - 0.112


In [4]:
data['tvt2'] = pd.Series(tvt)
data.head()

tweet_id                                         tweet_text  \
0  731166399389962242  🔥ca kkk grand wizard 🔥 endorses @hillaryclinto...   
1  714598641827246081  an open letter to trump voters from his top st...   
2  691809004356501505  america is a nation of second chances —@potus ...   
3  693204708933160960  brandon marshall visits and offers advice, sup...   
4  551099691702956032  rip elly may clampett: so sad to learn #beverl...   

        label       tvt  cv_fold        tt        tvt2  
0  unverified  training        1  training    testting  
1  unverified  training        1      test  validation  
2   non-rumor  training        2  training    training  
3   non-rumor  training        1  training    training  
4        true  training        3  training  validation

In [5]:
label_count = data['label'].value_counts()
label_count

unverified    374
non-rumor     374
true          372
false         370
Name: label, dtype: int64

In [6]:
combination = data.apply(lambda row: f"{row['label']}_{row['tvt2']}", axis=1).value_counts()
comparison = {}
for k, comb in combination.items():
    cv_fold = k.split("_")[1]
    label = k.split("_")[0]

    if cv_fold not in comparison:
        comparison[cv_fold] = {}
    
    comparison[cv_fold][label] = comb

labels = data['label'].unique().tolist()

def label_ratio(label_dict, labels):
    total = sum([v for k, v in label_dict.items()])
    
    report = ""
    for l in labels:
        report += f"{round(label_dict[l]/total, 2)}\t"
    
    return report[:-1]

def label_raw_value(label_dict, labels):
    total = sum([v for k, v in label_dict.items()])
    
    report = ""
    for l in labels:
        report += f"{label_dict[l]}\t"
    
    return report[:-1]

labels_str = ','.join(labels)
print(f"\nLabel,{labels_str}")
print(f"Original\t{label_ratio(label_count, labels)}")
for cv, comp in comparison.items():
    print(f"{cv.title()}\t{label_ratio(comp, labels)}")
    
labels_str = ','.join(labels)
print(f"\nLabel,{labels_str}")
print(f"Original\t{label_raw_value(label_count, labels)}")
for cv, comp in comparison.items():
    print(f"{cv.title()}\t{label_raw_value(comp, labels)}")


Label,unverified,non-rumor,true,false
Original	0.25	0.25	0.25	0.25
Training	0.24	0.26	0.24	0.26
Validation	0.29	0.24	0.25	0.23
Testting	0.25	0.25	0.28	0.23

Label,unverified,non-rumor,true,false
Original	374	374	372	370
Training	234	252	240	255
Validation	99	81	85	77
Testting	41	41	47	38


In [8]:
data.to_csv("../../data/processed/twitter15_dataset_with_tvt.csv", index=False)